## Setup and Data Import

In [1]:
import sys
sys.path.insert(0, '..')

from joblib import load

import Sita_Functions as fxns
from Sita_Functions import np, pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import timedelta

## Claims DF

In [2]:
# !python Sita_Preprocessing.py

In [3]:
claims = load('claims.pkl')

### New Columns

In [4]:
claims['AgeAtService'] = ( (claims.ClaimStartDt - claims.DOB)
                           / timedelta(days=365) ).astype(int)

claims['HasDied'] = claims.DOD.notna()
claims.loc[claims.HasDied == True, 'HasDied'] = 1
claims.loc[claims.HasDied == False, 'HasDied'] = 0
claims.HasDied = claims.HasDied.astype('category')

claims['ClaimCost'] = claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid

claims['IPDuration'] = (claims.DischargeDt - claims.AdmissionDt).dt.days

claims['ClaimDuration'] = (claims.ClaimEndDt - claims.ClaimStartDt).dt.days

### Variables

In [5]:
numeric_cols     = fxns.cols_by_dtype(claims)[0]
categorical_cols = fxns.cols_by_dtype(claims)[1]
date_cols        = fxns.cols_by_dtype(claims)[2]

physician_cols = claims.columns[claims.columns.str.contains('Physician')].to_list()
chronic_cols   = claims.columns[claims.columns.str.contains('Chronic')].to_list()
diagnosis_cols = claims.columns[claims.columns.str.contains('Diagnosis')].to_list()
procedure_cols = claims.columns[claims.columns.str.contains('Procedure')].to_list()

In [6]:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

### Summary Data

In [7]:
groupby_providers = claims.groupby('Provider')

In [8]:
mean_by_provider  = groupby_providers.mean().add_suffix('_mean').reset_index()

In [9]:
sum_by_provider   = groupby_providers.sum().add_suffix('_sum').reset_index()

In [10]:
chronic_by_provider = claims[['Provider'] + chronic_cols].groupby('Provider').sum().reset_index()

In [11]:
# need to get unique counts
count_by_provider = groupby_providers[
    ['BeneID', 'ClaimID', 'County', 'State', 'IPDuration']
    + physician_cols + diagnosis_cols + procedure_cols
    ].count().add_suffix('_count').reset_index()

In [12]:
providers = pd.merge(mean_by_provider, sum_by_provider, on='Provider')
providers = pd.merge(providers, count_by_provider, on='Provider')
providers = pd.merge(providers, chronic_by_provider, on='Provider')
providers.sort_index(axis=1, inplace=True)

In [13]:
print(providers.shape, '\n')
print(claims.columns.sort_values(), '\n')
print(providers.columns)

(5410, 62) 

Index(['AdmissionDt', 'AdmissionDt_Week', 'AgeAtService', 'Alzheimers_Chronic',
       'AttendingPhysician', 'BeneID', 'Cancer_Chronic', 'ClaimCost',
       'ClaimDuration', 'ClaimEndDt', 'ClaimEndDt_Week', 'ClaimID',
       'ClaimStartDt', 'ClaimStartDt_Week', 'ClmAdmitDiagnosisCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_10', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'County', 'DOB', 'DOD', 'DeductibleAmtPaid',
       'Depression_Chronic', 'Diabetes_Chronic', 'DiagnosisGroupCode',
       'DischargeDt', 'DischargeDt_Week', 'Gender', 'HasDied',
       'HeartFailure_Chronic', 'IPAnnualDeductibleAmt',
       'IPAnnualReimbursementAmt', 'IPDuration', 'InscClaimAmtReimburs

In [14]:
# Can't use, but have IPDuration count/mean/sum:
#     'AdmissionDt', 'AdmissionDt_Week', 'DischargeDt', 'DischargeDt_Week'

# Can't use, but have ClaimDuration count/mean/sum:
#     'ClaimEndDt', 'ClaimEndDt_Week', 'ClaimStartDt', 'ClaimStartDt_Week',

# Can't use, but have AgeAtService mean/sum and HasDied sum/ratio: 'DOB', 'DOD'

## New Columns

### Beneficiaries

In [15]:
def binary_ratios(df, col, group1, group2):
    by_provider = df.groupby(['Provider', col]).ClaimID.count().reset_index()

    colname = pd.DataFrame()
    colname[group1] = by_provider[by_provider[col] == 1].ClaimID.to_list()
    colname[group2] = by_provider[by_provider[col] == 0].ClaimID.to_list()
    
    newcolname1 = f'{group1}_ratio'
    newcolname2 = f'{group2}_ratio'
    
    providers[newcolname1] = round((colname[group1] / (colname[group1] + colname[group2])) * 100, 2)
    providers[newcolname2]  = round((colname[group2] / (colname[group1] + colname[group2])) * 100, 2)

binary_ratios(claims, 'Gender', 'GenderZero', 'GenderOne')
binary_ratios(claims, 'IsOutpatient', 'Outpatient', 'Inpatient')
binary_ratios(claims, 'PotentialFraud', 'PotentialFraud', 'NotPotentialFraud')
binary_ratios(claims, 'RenalDisease', 'RenalDisease', 'NotRenalDisease')
binary_ratios(claims, 'HasDied', 'HasDied', 'NotDead')

In [16]:
race_by_provider = claims.groupby(['Provider', 'Race']).ClaimID.count().reset_index()

race = pd.DataFrame()
race['race1'] = race_by_provider[race_by_provider.Race == 1].ClaimID.to_list()
race['race0'] = race_by_provider[race_by_provider.Race == 0].ClaimID.to_list()
race['race3'] = race_by_provider[race_by_provider.Race == 3].ClaimID.to_list()
race['race5'] = race_by_provider[race_by_provider.Race == 5].ClaimID.to_list()

providers['RaceZero_ratio']  = round((race.race0 / np.sum(race, axis=1) * 100), 2)
providers['RaceOne_ratio']   = round((race.race1 / np.sum(race, axis=1) * 100), 2)
providers['RaceThree_ratio'] = round((race.race3 / np.sum(race, axis=1) * 100), 2)
providers['RaceFive_ratio']  = round((race.race5 / np.sum(race, axis=1) * 100), 2)

In [17]:
ratio_cols = providers.columns[providers.columns.str.contains('_ratio')].to_list()
providers[['Provider'] + ratio_cols].sample(5)

,Provider,GenderZero_ratio,GenderOne_ratio,Outpatient_ratio,Inpatient_ratio,PotentialFraud_ratio,NotPotentialFraud_ratio,RenalDisease_ratio,NotRenalDisease_ratio,HasDied_ratio,NotDead_ratio,RaceZero_ratio,RaceOne_ratio,RaceThree_ratio,RaceFive_ratio
2665,PRV54317,48.00,52.00,100.0,0.0,0.0,100.0,20.00,80.00,4.00,96.00,20.00,80.00,0.00,0.00
5371,PRV57721,0.00,100.00,100.0,0.0,0.0,100.0,33.33,66.67,0.00,100.00,0.00,100.00,0.00,0.00
522,PRV51655,0.00,100.00,100.0,0.0,0.0,100.0,0.00,100.00,0.00,100.00,0.00,66.67,33.33,0.00
1257,PRV52563,39.11,60.89,100.0,0.0,0.0,100.0,22.41,77.59,0.21,99.79,13.11,86.05,0.85,0.00
4796,PRV57011,64.71,35.29,100.0,0.0,0.0,100.0,17.65,82.35,0.00,100.00,5.88,88.24,0.00,5.88


### Doctors

### Codes

### Money

### Time

### Location